# 問題1： 
#### 向我们总结此项目的目标以及机器学习对于实现此目标有何帮助。作为答案的部分，提供一些数据集背景信息以及这些信息如何用于回答项目问题。你在获得数据时它们是否包含任何异常值，你是如何进行处理的？【相关标准项：“数据探索”，“异常值调查”】

## A:
1. 透過boxplot，可以看出有異常值
2. 利用scatter，用EDA方式判斷是不是，極度異常，就此抓出了**TOTAL**
    - 把這欄位刪除，因為這是總合，並不是個體，不能當作poi
3.  **other** 存在THE TRAVEL AGENCY IN THE PARK
    - 選擇把欄位刪除，理由同上，這不屬於個體
4. 檢視最多NaN欄位，找到'LOCKHART EUGENE E'（全部特徵為NaN）。


# 問題2： 
#### 你最终在你的 POI 标识符中使用了什么特征，你使用了什么筛选过程来挑选它们？你是否需要进行任何缩放？为什么？作为任务的一部分，你应该尝试设计自己的特征，而非使用数据集中现成的——解释你尝试创建的特征及其基本原理。（你不一定要在最后的分析中使用它，而只设计并测试它）。在你的特征选择步骤，如果你使用了算法（如决策树），请也给出所使用特征的特征重要性；如果你使用了自动特征选择函数（如 SelectBest），请报告特征得分及你所选的参数值的原因。【相关标准项：“创建新特征”、“适当缩放特征”、“智能选择功能”】

## A:
### 1. 最終使用的特徵:
`
'retention_incentives/key_payments','bonus/salary','poi_emailratio',
'exercised_stock_options/salary','key_payments','retention_incentives'
 `

### 2. 篩選方式：
    - 採用RandomForest Feature Importance
![Imgur](http://i.imgur.com/3FzswNa.png?1)

### 3. 是否縮放？為什麼？
    - 使用`MinMaxScaler`，因為部分數據，尤其是異常數值跟中位數級距差距過大，為了避免這些異常數值主宰整個數據，所以進行縮放。

### 4. 創建特徵：
    - 部分特徵加總，是為了再創在新特徵所用
    - key_payments ＝salary + bonus + others
    - retention_incentives ＝ long_term_incentive + total_stock_value  
    - bonus/salary
    - exercised_stock_options/salary
    - retention_incentives/key_payments
    - messagetotal = from_messages + to_messages 
    - poitotal = from_this_person_to_poi + from_poi_to_this_person
    - poi_emailratio = poitotal/messagetotal



# 問題3：  
#### 你最终使用了什么算法？你还尝试了其他什么算法？不同算法之间的模型性能有何差异？【相关标准项：“选择算法”】

### A:
最終使用集成法Ensemble**VotingClassifier** 多分類器，多數決策。

 `from sklearn.ensemble import VotingClassifier`
 
**分類器包含以下：**
1. DecisionTreeClassifier [官方文檔](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)
2. RandomForestClassifier [官方文檔](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)
3. GradientBoostingClassifier [官方文檔](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier)
4. AdaBoostClassifier [官方文檔]()

# 問題4 :
#### 调整算法的参数是什么意思，如果你不这样做会发生什么？你是如何调整特定算法的参数的？（一些算法没有需要调整的参数 – 如果你选择的算法是这种情况，指明并简要解释对于你最终未选择的模型或需要参数调整的不同模型，例如决策树分类器，你会怎么做）。【相关标准项：“调整算法”

## A:

### 調整算法參數：
調整算法參數，會改變輸出結果，像是Learning Rate，過大容易發散，太小不容易收斂。如果不調整參數，算法表現就無法得到進一步優化。

### 如何調整算法參數：
使用以下的庫GridSearchCV，去做收尋
`from sklearn.model_selection import GridSearchCV`

[GradientBoostingClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier)。

1. learning_rate
2. max_depth
3. min_samples_split
4. subsample

### 怎麼做：
1. 使用GridSearch 先針對，單一參數，進行大範圍測試
    - `learning_rate : [0.1, 0.01, 0.001, 0.0001]`
    
2. 找出最優的範圍後，再針對其他參數，進行以上動作，範圍縮小進行，排列組合比對出最優的參數。

    `
    {'learning_rate':[1/(10**i) for i in range(1,4)],
    'max_depth':[2,3,4,5],
    'min_samples_split':[2,3,4,5],
    'subsample':[i/10 for i in range(7,10)]}
    `
3. 調參後，驗證測試數據，表現優，就停止調參。

4. 調好參數好，把該算法，再放入多數決策模型`VotingClassifier`

# 問題5 :
什么是验证，未正确执行情况下的典型错误是什么？你是如何验证你的分析的？【相关标准项：“验证策略”】

## A:
1. 驗證，就是把數據，拆開成，訓練跟測試兩個集合，訓練樣本又再分割出，交叉驗證集合。
2. 最典型的錯誤就是，把訓練過的數據，拿去做預測，容易造成overfitting。
    - 訓練過的數據表現不好，就是underfitting
3. 我把原數據依據8:2拆開，再把訓練集，隨機打散成五份，取其中一份當作交叉驗證，如果表現差異不多，就沒有overfitting問題。

## 問題 6:
#### 给出至少 2 个评估度量并说明每个的平均性能。解释对用简单的语言表明算法性能的度量的解读。【相关标准项：“评估度量的使用”】

## A:
我使用Precision，Recall當作評估度量，但是這很難會有好的評估。

- True Positive(TP)： 是Poi，被成功預測為Poi
- False Positive(FP)： 非Poi，被預測為Poi
- True Negative(TN)： 非Poi，被成功預測為非Poi
- False Negative(FN)： 是Poi，被預測為非Poi

- 精確率Precision: TP / (TP + FP)，精確率代表的是：在所有被分類為正例的樣本中，真正是正例的比例。

- 召回率Recall: TP/(TP + FP)，在所有實際為正例的樣本中，被預測為正例的樣本比例，簡單說就是「總共這麼多1，你預測了多少？」

- F1 定義 : 2.0 * TP/(2*TP + FP + FN)